#  План семинара

1. Линейный классификатор в задаче бинарной классификации
2. Кодирование категориальных признаков

## Задача бинарной классификации

### Логистическая регрессия

y = {-1, 1}

$b(x) = \sigma(<w,x>)$,

где $\sigma(z) = \frac{1}{1 + e^{-z}}$

То есть, мы предсказываем $P(y_i = 1| X_i)$ - вероятность того, что наблюдение принадлежит классу +1

Обучаем с помощью функционала: Максимального лог правдоподобия (флэшбек из статистики)

$Q(w) = -\Sigma_{i=0}^{n}(y_i*log(b(x_i)) + (1 - y_i)log(1 - b(x_i))) \rightarrow min_w$



In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
np.random.seed(42)

In [3]:
data = pd.read_csv(
    'https://raw.githubusercontent.com/evgpat/hse_ml_light_autumn_2023/main/datasets/bike_buyers_clean.csv'
)

HTTPError: HTTP Error 404: Not Found

In [ ]:
data

# Обзор данных

In [ ]:
# проверим типы колонок в датасете
data.dtypes

In [ ]:
X = data.iloc[:,:-1]
X.drop(columns='ID', inplace=True)

y = data['Purchased Bike']

In [ ]:
y.value_counts()

In [ ]:
num_cols = X.columns[X.dtypes == 'int64'].tolist()
cat_cols = X.columns[X.dtypes == 'object']

print(f"We have {len(num_cols)} numeric columns: {', '.join(num_cols)}")
print(f"And {len(cat_cols)} categorical columns: {', '.join(cat_cols)}")

In [ ]:
for col in cat_cols:
    print(col)
    display(X[col].value_counts(normalize=True))
    print()

In [ ]:
# у нас есть категориальные переменные разных видов!

binary_cols = cat_cols[X[cat_cols].nunique() == 2].tolist()
ordinal_cols = ['Commute Distance', 'Education']
cat_cols = cat_cols.difference(binary_cols + ordinal_cols).tolist()

In [ ]:
for col in num_cols:
    print(col)
    display(X[col].describe())
    print()

In [ ]:
X.describe()

In [ ]:
# classes are balanced !
y.value_counts(normalize=True)

In [ ]:
# transform y to numeric column
y = (y == 'Yes').astype(int)
y

# Подготовка данных

## Кодирование категориальных признаков

In [ ]:
# run if not installed yet

!pip install category_encoders

In [ ]:
from category_encoders.ordinal import OrdinalEncoder # LabelEncoder
from category_encoders.one_hot import OneHotEncoder # OneHotEncoding
from category_encoders.target_encoder import TargetEncoder # счетчики+сглаживание

In [ ]:
X['Education'].unique()
dict_our = {'Bachelors':2, 'Partial College':1}
X['Education'].map(dict_our)

In [ ]:
# Ordinal: from categories to numbers

ord_enc = OrdinalEncoder()
ord_enc.fit_transform(X['Education'])

In [ ]:
# One hot: from k categories to k dummy columns

one_hot_enc = OneHotEncoder()

one_hot_enc.fit_transform(X['Education'], drop='first')
# * fit -> определить количество новых столбцов (по кол-ву категорий)
# * transform -> создать новые столбцы
# * fit_transform = fit + transform

# Нужно ли удалять какую-то из колонок после такого кодирования ?


Target encoding вычисляет значения по формуле

$$\frac{mean(target)\cdot n_{rows} + \alpha \cdot globalMean}{n_{rows} + \alpha} $$

In [ ]:
data.columns

In [ ]:
df = X.copy()
df['y'] = y

In [ ]:
df.groupby('Education')['y'].mean()

In [ ]:
df[['Education', 'y']].head()

In [ ]:
# target encoding: from k categories to posterior probabilites of y == 1 - P(y==1 | category == c1)

tgt_enc = TargetEncoder(smoothing=1)

# smoothing - это коэффициент сглаживания alpha, чем он больше, тем больше регуляризация

tgt_enc.fit_transform(X['Education'], y)

In [ ]:
# энкодер можно применять сразу на весь датафрейм

tgt_enc = TargetEncoder(cols=['Education', 'Gender', 'Region'])
tgt_enc.fit_transform(X, y)

Помимо сглаживания, для борьбы с переобучением при таргет энкодинге в лекции предлагались и другие методы

- Добавление случайного шума
- Вычисление счетчиков на кросс-валидации
- Expanding mean encoding

Первые две идеи реализованы в классе LeaveOneOut

- значения считаются на основе кросс-валидации вида leave one out (то есть значение энкодинга для конкретного наблюдения будет считаться по всем наблюдениям, кроме этого)
- параметр sigma отвечает за дисперсию случайного шума, который добавляется к значению энкодинга (чем больше sigma, тем больше регуляризация)

In [ ]:
from category_encoders.leave_one_out import LeaveOneOutEncoder

loo_enc = LeaveOneOutEncoder(sigma=3.)

loo_enc.fit_transform(X['Education'], y)

## Масштабирование числовых признаков

In [ ]:
X['Income'].hist()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit_transform(X[['Income']])

Есть две проблемы:
- класc StandardScaler не умеет работать только на части колонок датафрейма
- классы sklearn возвращают numpy arrays, а не pandas dataframe, что не удобно

In [ ]:
num_cols

In [ ]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([('scaler', StandardScaler(), num_cols)], remainder='passthrough') # 'drop'

In [ ]:
ct.fit_transform(X)

In [ ]:
# нет удобной реализации - напишем сами !

from sklearn.base import TransformerMixin

class CustomScaler(TransformerMixin):
    def __init__(self, cols, scaler=None):
        self.cols = cols
        self.scaler = scaler or StandardScaler()
        
    def fit(self, X, y=None):
        num_cols = X.copy()[self.cols]
        self.scaler.fit(num_cols)
        return self
    def transform(self, X, y=None):
        X_res = X.copy()
        num_cols_tr = self.scaler.transform(X_res[self.cols])
        for i, col in enumerate(self.cols):
            X_res[col] = num_cols_tr[:,i]
        return X_res

In [ ]:
sc = CustomScaler(num_cols)
X2 = sc.fit_transform(X)

In [ ]:
X2.info()

# Соберем все преобразования данных в pipeline

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

p1 = Pipeline([
    ('ordinal_encoder_', OrdinalEncoder(cols=ordinal_cols + binary_cols + cat_cols)), # плохо!!!
    ('scaler_', CustomScaler(num_cols)),
    ('model_', LogisticRegression())
    ])

p2 = Pipeline([
    ('one_hot_encoder_', OneHotEncoder(cols=ordinal_cols + binary_cols+cat_cols)),
    ('scaler_', CustomScaler(num_cols)),
    ('model_', LogisticRegression())
    ])

p3 = Pipeline([
    ('target_encoder_', TargetEncoder(cols=ordinal_cols + binary_cols+cat_cols)),
    ('scaler_', CustomScaler(num_cols)),
    ('model_', LogisticRegression())
])

p4 = Pipeline([
    ('ordinal_encoder_', OrdinalEncoder(cols=ordinal_cols)),
    ('one_hot_encoder_', OneHotEncoder(cols=binary_cols+cat_cols)),
    ('scaler_', CustomScaler(num_cols)),
    ('model_', LogisticRegression())
    ])

p5 = Pipeline([
    ('ordinal_encoder_', OrdinalEncoder(cols=ordinal_cols)),
    ('one_hot_encoder_', OneHotEncoder(cols=binary_cols)),
    ('target_encoder_', TargetEncoder(cols=cat_cols)),
    ('scaler_', CustomScaler(num_cols)),
    ('model_', LogisticRegression())
])

p6 = Pipeline([
    ('one_hot_encoder_', OneHotEncoder(cols=binary_cols)),
    ('target_encoder_', TargetEncoder(cols=cat_cols + ordinal_cols)),
    ('scaler_', CustomScaler(num_cols)),
    ('model_', LogisticRegression())
])

In [ ]:
cat_cols

In [ ]:
# пример работы с пайплайном
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y)

p1.fit(X_train, y_train)

#print(p1)

y_pred = p1.predict(X_test)

print(accuracy_score(y_test, y_pred))

In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import cross_val_score
X, y = datasets.load_iris(return_X_y=True)
clf = LogisticRegression()
cross_val_score(clf, X, y, cv=5, scoring='accuracy')

# Сравнение качества классификации при разных пайплайнах преобразования данных

Вообще существует довольно большое количество метрик для задачи бинарной классификации (о них будет подробно рассказано на лекциях)

Но для нашей задачи разберем самую простую и интуитивную метрику: accuracy

$accuracy = \frac{1}{n}\Sigma_{i=0}^n [\hat y_i == y_i]$

То есть доля правильных предсказаний

In [ ]:
from sklearn.model_selection import cross_validate, cross_val_score
import warnings

warnings.filterwarnings('ignore')

cross_val_score()

In [ ]:
for i, pipe in enumerate([p1, p2, p3, p4, p5, p6]):
    cv_res = cross_validate(pipe,
                            X,
                            y,
                            cv=5,
                            scoring='accuracy'
                           )
    print(f"Pipeline {i + 1}: mean cv accuracy = {cv_res['test_score'].mean()}")

In [ ]:
for i, pipe in enumerate([p1, p2, p3, p4, p5, p6]):
    cv_res = cross_val_score(pipe,
                            X,
                            y,
                            cv=5,
                            scoring='accuracy'
                           )
    print(f"Pipeline {i + 1}: mean cv accuracy = {cv_res['test_score'].mean()}")